In [15]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import regex as re
import nltk
from nltk.corpus import stopwords
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn import preprocessing

In [9]:
TRAIN_PATH = 'train_dataset_RUTUBE/'
MODELS_PATH = 'doc2vec_models'

In [2]:
data =  pq.read_table(TRAIN_PATH+'videos.parquet').to_pandas()

In [78]:
# schema = pa.schema([
#     ('item_id', pa.string()),
#     ('video_title', pa.string()),
#     ('author_title', pa.string()),
#     ('tv_title', pa.string()),
#     ('season', pa.int16()),
#     ('video_description', pa.string()),
#     ('category_title', pa.string()),
#     ('publicated', pa.string()),
#     ('duration', pa.int32()),
#     ('channel_sub', pa.int32()),
#     ('tv_sub', pa.int32()),
#     ('CTR_10days_21_07', pa.float32()),
#     ('CTR_10days_01_08', pa.float32()),
#     ('CTR_10days_10_08', pa.float32()),
#     ('CTR_10days_21_08', pa.float32()),
#     ('__index_level_0__', pa.int64())
# ])
#
# source_table = pa.Table.from_pandas(data)
# source_table = source_table.cast(schema)
# pq.write_table(source_table, 'my_parquet_table.parquet')

In [3]:
data.rename(columns={'ctr.CTR_10days_21_07': 'CTR_10days_21_07', 'ctr.CTR_10days_01_08': 'CTR_10days_01_08',
                     'ctr.CTR_10days_10_08': 'CTR_10days_10_08', 'ctr.CTR_10days_21_08': 'CTR_10days_21_08'}, inplace=True)

In [2]:
users = pq.read_table('train_dataset_RUTUBE/player_starts_train.parquet').to_pandas()

In [19]:
small_users = pd.read_csv('train_dataset_RUTUBE/small_player_starts_train.csv')

In [20]:
authorized = users[users['is_autorized'] == 1]
authorized.drop('is_autorized', inplace=True, axis=1)

/var/folders/vn/wf1rpzp51jj_bp2lz71rs3q80000gn/T/ipykernel_9919/1923230411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authorized.drop('is_autorized', inplace=True, axis=1)


In [37]:
def preprocess_xtext(text: str, remove_stopwords: bool) -> str:
    text = re.sub(r"http\S+", "", text)
    text = re.sub("[^А-Яа-я0-9 ]+", " ", text)
    tokens = nltk.word_tokenize(text)
    if remove_stopwords:
        tokens = [w.lower().strip() for w in tokens if not w.lower() in stopwords.words("russian")]
        return tokens
    return tokens

In [4]:
import string
translator = str.maketrans('', '', string.punctuation)

'''Making text embeddings'''


data['video_title'] = data['video_title'].apply(lambda x: x.translate(translator).lower().strip() if isinstance(x, str) else x)
data['video_description'].fillna("", inplace=True)
data['video_description'] = data['video_description'].apply(lambda x: x.translate(translator).lower().strip() if isinstance(x, str) else x)


In [62]:
# video_descriptions = [TaggedDocument(doc, [i]) for i, doc in enumerate(data['video_description'].tolist())]
author_titles = [TaggedDocument(doc, [i]) for i, doc in enumerate(data['author_title'].tolist())]
video_titles = [TaggedDocument(doc, [i]) for i, doc in enumerate(data['video_title'].tolist())]

# model_video_descr = Doc2Vec(video_descriptions,
#                 vector_size=100,
#                 window=len(video_descriptions),
#                 workers=8,
#                 epochs=1,
#                 seed=42)

model_author_titles = Doc2Vec(author_titles,
                vector_size=100,
                window=len(author_titles),
                workers=8,
                epochs=10,
                seed=42)

model_video_titles = Doc2Vec(video_titles,
                vector_size=100,
                window=len(video_titles),
                workers=8,
                epochs=10,
                seed=42)

KeyboardInterrupt: 

In [ ]:
model_video_titles.save(MODELS_PATH+'model_video_titles')
model_author_titles.save(MODELS_PATH+'model_author_titles')
# model_video_descr.save(MODELS_PATH+'model_video_descr')


In [84]:
# author_titles = np.load('doc2vec_models/model_author_titles.dv.vectors.npy', allow_pickle=True)
video_titles = np.load('doc2vec_models/model_video_titles.dv.vectors.npy', allow_pickle=True)

In [85]:
text_embs = np.concatenate([author_titles,video_titles], axis=-1)

ValueError: zero-dimensional arrays cannot be concatenated

In [86]:
text_embs_normalized = preprocessing.minmax_scale(video_titles.T).T

In [99]:
df = pd.DataFrame(text_embs_normalized, columns = list(map(str, list(range(100)))))

In [100]:
df['item_id'] = data.item_id

In [101]:
categories_dict = {cat: idx for idx, cat in enumerate(data['category_title'].unique())}
data['category_title'] = data['category_title'].map(categories_dict)

author_titles = {title: idx for idx, title in enumerate(data['author_title'].unique())}
data['author_title'] = data['author_title'].map(author_titles)

In [102]:
cols = ['item_id']
cols.extend(list(map(str, list(range(100)))))

In [103]:
df = df[cols]

In [104]:
final = df.merge(data,how='inner', on='item_id')

In [97]:
final.drop(['video_title', 'tv_title', 'season', 'video_description',
       'channel_sub', 'tv_sub',
       ], inplace=True, axis=1)

In [105]:
final['CTR_10days_21_07'].fillna(data['CTR_10days_21_07'].mode()[0], inplace=True)
final['CTR_10days_01_08'].fillna(data['CTR_10days_01_08'].mode()[0], inplace=True)
final['CTR_10days_10_08'].fillna(data['CTR_10days_10_08'].mode()[0], inplace=True)
final['CTR_10days_21_08'].fillna(data['CTR_10days_21_08'].mode()[0], inplace=True)

In [106]:
final['publicated'] = pd.to_datetime(final['publicated'])

In [107]:
final['upld_year'] = final['publicated'].apply(lambda x: x.year)
final['upld_month'] = final['publicated'].apply(lambda x: x.month)
final['upld_day'] = final['publicated'].apply(lambda x: x.day)
final['upld_hour'] = final['publicated'].apply(lambda x: x.hour)
final['upld_minute'] = final['publicated'].apply(lambda x: x.minute)
final['upld_second'] = final['publicated'].apply(lambda x: x.second)
final['upld_dayofweek'] = final['publicated'].apply(lambda x: x.weekday())

In [108]:
final

,item_id,0,1,2,3,4,5,6,7,8,...,CTR_10days_01_08,CTR_10days_10_08,CTR_10days_21_08,upld_year,upld_month,upld_day,upld_hour,upld_minute,upld_second,upld_dayofweek
0,video_165654,0.317030,0.583350,0.319950,0.233488,0.397311,0.335484,0.683385,0.544738,0.459705,...,0.0,0.000000,0.00,2022,12,8,13,53,5,3
1,video_1173704,0.481264,0.387798,0.602036,0.802317,0.655285,0.626316,0.562532,0.271270,0.515960,...,0.0,0.000000,0.00,2022,3,24,9,19,15,3
2,video_23927,0.648496,0.440843,0.160715,0.613938,0.367600,0.542256,0.304734,0.719216,0.541566,...,0.0,0.000000,0.00,2022,3,19,17,41,49,5
3,video_1003780,0.203357,0.307920,0.590847,0.516022,0.755484,0.283917,0.717133,0.625334,0.525521,...,1.0,0.000000,0.00,2021,2,20,11,50,53,5
4,video_105383,0.744297,0.835887,0.621679,0.773973,0.504829,0.539626,0.493591,0.560223,0.590560,...,0.0,0.000000,0.00,2023,8,11,9,2,7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320657,video_839160,0.292854,0.666169,0.771124,0.671644,0.643341,0.410232,0.162509,0.522382,0.561864,...,0.0,0.333333,0.25,2017,9,11,22,0,13,0
2320658,video_2134427,0.557534,0.428624,0.511768,0.810072,0.586449,0.477702,0.389940,0.329786,0.467130,...,0.0,0.000000,0.00,2022,3,28,11,19,33,0
2320659,video_1453482,0.434726,0.334742,0.281761,0.154127,0.254287,0.386525,0.505897,0.527292,0.430226,...,0.0,0.000000,0.00,2022,9,10,20,5,17,5
2320660,video_2225207,0.805957,0.767042,0.671743,0.632880,0.815049,0.631796,0.608704,0.433524,0.722830,...,0.0,0.000000,0.00,2023,7,6,12,16,29,3


In [109]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(final, test_size=0.2, random_state=42)

In [112]:
import pynndescent
index = pynndescent.NNDescent(train.drop(['item_id', 'video_description', 'tv_title', 'video_title', 'CTR_10days_01_08', 'CTR_10days_21_07', 'CTR_10days_10_08', 'CTR_10days_21_08', 'tv_sub', 'season', 'publicated'], axis=1), n_jobs=8, n_neighbors=5)

In [113]:
index.prepare()

/Users/gd/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [179]:
neighbors = index.query(test.drop(['item_id', 'video_description', 'tv_title', 'video_title', 'CTR_10days_01_08', 'CTR_10days_21_07', 'CTR_10days_10_08', 'CTR_10days_21_08', 'tv_sub', 'season', 'publicated'], axis=1).head(10), k=10)

In [184]:
test.head(10)['item_id']

1025987     video_594977
468859     video_2000944
1638170     video_947196
423766     video_2283492
1597720    video_1537558
1698657      video_53898
67347      video_1350571
872393      video_468461
1564649    video_2155498
79079       video_687512
Name: item_id, dtype: object

In [188]:
for i in neighbors[0][9]:
    print(data[data['item_id'] == train.iloc[i].item_id].video_title)


1672977    Угадай мелодию, 1999. Часть 1
Name: video_title, dtype: object
383969    Испытание Восьмикрута HD600 на пилораме.mp4
Name: video_title, dtype: object
1539480    Febspot платит за просмотры и рефералы - полны...
Name: video_title, dtype: object
152574    МАКИЯЖ БЮДЖЕТНОЙ КОСМЕТИКОЙ из Золотого Яблока...
Name: video_title, dtype: object
2213523    Бакалавриат 2+2_ условия осмысленности – Андре...
Name: video_title, dtype: object
729366    Рыбалка, хутор Новторицкий. Варнавинский сброс...
Name: video_title, dtype: object
2262544    ROBLOX| мимика
Name: video_title, dtype: object
955716    Как переделать диски МТЗ, на диски ГАЗ 53 в до...
Name: video_title, dtype: object
529950    Архив гематолога. Выпуск 5. Криптококковый мен...
Name: video_title, dtype: object
1823857    Мастер класс. Гуашь. РЯБИНОВЫЕ БУСЫ
Name: video_title, dtype: object


In [189]:
data[data['item_id'] == 'video_2155498'].video_title

1564647    Вестница весны - под музыку, для любителей про...
Name: video_title, dtype: object